In [2]:
import requests # module to download webpage
import pandas as pd # this module helps with pandas
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import geocoder # import geocoder
print('Imports Done')

Imports Done


<h2> Question 1 </h2>

<h3> Downloading Boroughs from Wiki </h3>

In [3]:
html_data = requests.get(r'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(html_data, 'html5lib')

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
#Check for any null values
df.isnull().values.any()

False

In [8]:
#Checking the SHape of the Final Dataframe
df.shape

(103, 3)

Shape is 103 rows and 3 columns 

<h2> Question 2 </h2>

<H3> Using file shared in Coursera Link since dgeocoder taking too much time </H3>

In [9]:
data = pd.read_csv("https://cocl.us/Geospatial_data")
data = data.rename(columns={'Postal Code': 'PostalCode'})
data

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


<H3> Both Tables have same number of rows. Inner Join is being used to combine them </H3>

In [10]:
df = df.join(data.set_index('PostalCode'), on='PostalCode', how='inner')
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<h3> Rendering Folium Map for Tooronto </h3>

In [11]:
import folium

from geopy.geocoders import Nominatim 
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# Creating the map of Toronto
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# adding markers to map
for latitude, longitude, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='green',
        fill=True
        ).add_to(map_Toronto)  
    
map_Toronto


<h2> Question 3 </h2>

<h3> Foursquare credentials </h3>

In [13]:
CLIENT_ID = 'RBCGHJHO4LR0DXDLMFI41X4KS4DTG3YGYFWH043D1JLM40AH' # your Foursquare ID
CLIENT_SECRET = 'VFC1ZZA4C42B0WL3C5340JCLP3RAA0K0MJSLYQSYO3FR01ER' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


<h1> Getting Nearby Venues from Four Square </h1>

Function for getting venues for a given name

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

FUnction to individually get venues for each name

In [14]:
Toronto_venue = getNearbyVenues(df['Neighborhood'], df['Latitude'], df['Longitude'])

Displaying the data set

In [15]:
Toronto_venue

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,Coffee Shop
...,...,...,...,...,...
1315,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Jim & Maria's No Frills,Grocery Store
1316,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,McDonald's,Fast Food Restaurant
1317,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,Tanning Salon
1318,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Once Upon A Child,Kids Store


<h1> Data wrangling for preparing data for k-means clustering same as Lab excercise </h1>

In [16]:
Toronto_venue.groupby('Venue Category').max()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Accessories Store,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,Ardene Shoes Outlet
Airport,Downsview East,43.737473,-79.394420,Toronto Downsview Airport (YZD)
Airport Food Court,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Billy Bishop Café
Airport Gate,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Gate 8
Airport Lounge,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,Porter Lounge
...,...,...,...,...
Warehouse Store,Thorncliffe Park,43.705369,-79.349372,Costco
Wine Bar,"Little Portugal, Trinity",43.653206,-79.400049,Paris Paris Bar
Wings Joint,Ontario Provincial Government,43.665860,-79.383160,Wingporium


In [17]:
Toronto_venue_OH = pd.get_dummies(Toronto_venue[['Venue Category']], prefix="", prefix_sep="")
Toronto_venue_OH

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1316,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1317,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1318,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
Toronto_venue_OH['Neighborhood'] = Toronto_venue['Neighbourhood']

In [19]:
toronto_grouped = Toronto_venue_OH.groupby('Neighborhood').mean().reset_index()

In [20]:
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h2> No of Top venues have been changed to 7 </h2>

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

import numpy as np

num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Clothing Store,Department Store,Electronics Store,Eastern European Restaurant
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Playground,Sandwich Place,Pub,Dim Sum Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Grocery Store,Mobile Phone Shop,Diner,Sandwich Place,Bridal Shop
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Department Store,Electronics Store,Eastern European Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Thai Restaurant,Juice Bar,Liquor Store,Restaurant


In [22]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Clothing Store,Department Store,Electronics Store,Eastern European Restaurant
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Playground,Sandwich Place,Pub,Dim Sum Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Grocery Store,Mobile Phone Shop,Diner,Sandwich Place,Bridal Shop
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Department Store,Electronics Store,Eastern European Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Thai Restaurant,Juice Bar,Liquor Store,Restaurant
...,...,...,...,...,...,...,...,...
94,Willowdale West,Grocery Store,Supermarket,Pharmacy,Coffee Shop,Butcher,Pizza Place,Airport Service
95,Woburn,Coffee Shop,Korean BBQ Restaurant,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore
96,Woodbine Heights,Athletics & Sports,Park,Beer Store,Skating Rink,Curling Ice,Department Store,Electronics Store
97,York Mills West,Park,Convenience Store,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Drugstore


<h1> K-means clustering similar to Lab excercise </h1>

In [23]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [24]:
# set number of clusters
k_num_clusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k_num_clusters, random_state=0).fit(toronto_grouped_clustering)
kmeans

KMeans(n_clusters=5, random_state=0)

In [25]:
kmeans.labels_[0:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       4, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 4, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 2])

In [26]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [27]:
toronto_merged = df

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Drugstore,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Intersection,Deli / Bodega,Eastern European Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0.0,Coffee Shop,Bakery,Park,Café,Breakfast Spot,Dessert Shop,Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Miscellaneous Shop,Coffee Shop,Women's Store
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Diner,Café,Italian Restaurant


In [28]:
toronto_merged_nonan = toronto_merged.dropna(subset=['Cluster Labels'])

In [29]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [30]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_nonan['Latitude'], toronto_merged_nonan['Longitude'], toronto_merged_nonan['Neighborhood'], toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters)
        
map_clusters

In [31]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 0, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
1,North York,0.0,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Intersection,Deli / Bodega,Eastern European Restaurant
2,Downtown Toronto,0.0,Coffee Shop,Bakery,Park,Café,Breakfast Spot,Dessert Shop,Restaurant
3,North York,0.0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Miscellaneous Shop,Coffee Shop,Women's Store
4,Queen's Park,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Diner,Café,Italian Restaurant
6,Scarborough,0.0,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,0.0,Café,Restaurant,Coffee Shop,Seafood Restaurant,Concert Hall,Bakery,Steakhouse
98,Etobicoke,0.0,River,Pool,Yoga Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
99,Downtown Toronto,0.0,Sushi Restaurant,Bubble Tea Shop,Café,Ramen Restaurant,Ethiopian Restaurant,Ice Cream Shop,Indian Restaurant
100,East Toronto Business,0.0,Light Rail Station,Auto Workshop,Skate Park,Brewery,Smoke Shop,Burrito Place,Farmers Market


<h2> CHange i to get information about the cluster </h2>

In [33]:
i = 1

print ("Cluster " + str(i))

toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == i-1, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

Cluster 1


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
1,North York,0.0,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Intersection,Deli / Bodega,Eastern European Restaurant
2,Downtown Toronto,0.0,Coffee Shop,Bakery,Park,Café,Breakfast Spot,Dessert Shop,Restaurant
3,North York,0.0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Miscellaneous Shop,Coffee Shop,Women's Store
4,Queen's Park,0.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Bar,Diner,Café,Italian Restaurant
6,Scarborough,0.0,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop
...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,0.0,Café,Restaurant,Coffee Shop,Seafood Restaurant,Concert Hall,Bakery,Steakhouse
98,Etobicoke,0.0,River,Pool,Yoga Studio,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run
99,Downtown Toronto,0.0,Sushi Restaurant,Bubble Tea Shop,Café,Ramen Restaurant,Ethiopian Restaurant,Ice Cream Shop,Indian Restaurant
100,East Toronto Business,0.0,Light Rail Station,Auto Workshop,Skate Park,Brewery,Smoke Shop,Burrito Place,Farmers Market


<h1> Base on the clusters above we can guess the clusters indicative of following </h1>
Cluster 1: Restuarants and Coffee Shops
Cluster 2: Parks and PLayground
Cluster 3: Only one data set seems outlier
Cluster 4: BAse Ball
Cluster 5: Only one data set seems outlier

I tried the excercise with only 7 most common venues. Cluster results dudn't chnage post that as well.